<a href="https://colab.research.google.com/github/IlariaSalogni/Quantification_VAs/blob/main/Hierarchical_Quantification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From Classification to Quantification: New Methods for Estimating Cause-of-Death Prevalence from Verbal Autopsies
This notebook contains the experiments illustrated in my master thesis, that can be found in [this Github repository](https://github.com/IlariaSalogni/Quantification_VAs/tree/main).

### Flow of the notebook
0. [Requirments](#requirments)
1. [Data preprocessing](#data_pre)
2. [Embeddings and train-test split](#train_test)
3. Models
  * [1° and 2° baselines](#1-2base)
  * [3° baseline](#3base)
  * [Hierarchical quantifier](#hier)

<a name="requirments"></a>
# 0. Requirments

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

!pip install QuaPy
import quapy as qp
from quapy.protocol import NPP
from quapy.data import LabelledCollection
from quapy.method.aggregative import CC, PCC, ACC, PACC, EMQ

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np

from transformers import AutoTokenizer, AutoModel
import torch

import time
import copy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for abstention: filename=abstention-0.1.3.1-py3-none-any.whl size=25422 sha256=cf516021180d1b842436677858710fc0d3921434fbd0bc392ad13cc6d948548f
  Stored in directory: /root/.cache/pip/wheels/db/c9/5d/304d0d55504c5bbde53ef42b55ce70d598873246429d6a28d0
Successfully built abstention


<a name="data_pre"></a>
# 1. Data preprocessing
Verbal Autopsies datasets were shared after NDA agreement and therefore can not be uploaded, but IMDB and 20newsgroup datasets can easily be fetched.

### Karonga HDSS dataset

In [ ]:
# IMPORTA I FILE, APRI CON PANDAS
karonga_hdss = pd.read_csv("/content/Karonga_HDSS_VA_ISNov23_val.csv")
print(karonga_hdss.shape) # 5105 rows × 282 columns

karonga_hdss.dropna(subset=["ucod2"], inplace=True)
print(karonga_hdss.shape) # 3810 rows × 282 columns

karonga_hdss.dropna(subset=["narrative"], inplace=True)
print(karonga_hdss.shape) # 3391 rows × 2 columns

labels = karonga_hdss["ucod2"].unique()
print("initial labels are", len(labels))

value_counts = karonga_hdss['ucod2'].value_counts()
values_to_remove = value_counts[value_counts <= 5].index
karonga_hdss = karonga_hdss[~karonga_hdss['ucod2'].isin(values_to_remove)]

labels = karonga_hdss["ucod2"].unique()
print("labels after taking out ones with 5 or less items are", len(labels))

(5105, 282)
(3810, 282)
(3391, 282)
initial labels are 67
labels after taking out ones with 5 or less items are 56


In [ ]:
def transform_and_split(value):
    # Convert value to integer
    integer_value = int(value)
    # Split integer into single numbers and join with '.'
    transformed_value = '.'.join(str(integer_value))
    return transformed_value

# Apply the transformation to the 'ucod2' column
karonga_hdss['labels'] = karonga_hdss['ucod2'].astype(int).apply(transform_and_split)
karonga_hdss["labels"]

0       4.3.0
1       3.6.2
2       3.1.1
6       3.1.2
7       3.1.2
        ...  
5096    1.1.0
5097    3.1.0
5098    1.4.2
5100    3.6.4
5102    3.0.0
Name: labels, Length: 3362, dtype: object

In [ ]:
# NARRATIVES ONLY SUBSET
krg_narratives = karonga_hdss[["narrative", "labels"]].copy()

value_counts = krg_narratives['labels'].value_counts()
values_to_remove = value_counts[value_counts <= 5].index
krg_narratives = krg_narratives[~krg_narratives['labels'].isin(values_to_remove)] #3362 rows × 2 columns

labels = krg_narratives["labels"].unique()
print("labels after taking out ones with 5 or less items are", len(labels)) # 56 labels left
krg_narratives.shape

labels after taking out ones with 5 or less items are 56


(3362, 2)

In [ ]:
# CONCATENATED
columns_to_keep.append("narrative")
krg_concatenated = karonga_hdss[columns_to_keep]

def concatenate_values(row):
    concatenated_values = []
    for column_name, content in row.items():
        concatenated_values.append(f"{column_name}_{content}")
    return ' '.join(concatenated_values)

krg_concatenated['concatenated'] = krg_concatenated.apply(concatenate_values, axis=1)

# aggiungi colonna labels
krg_concatenated["labels"] = krg_narratives["labels"]
krg_concatenated.dropna(subset=["labels"], inplace=True)

# 2. togli valori con meno di 5 occorrenze,  printa lunghezza dataset, printa unique labels
value_counts = krg_concatenated['labels'].value_counts()
values_to_remove = value_counts[value_counts <= 5].index
krg_concatenated = krg_concatenated[~krg_concatenated['labels'].isin(values_to_remove)] # 3362 rows × 265 columns

print(krg_concatenated.shape)

labels = krg_concatenated["labels"].unique()
print("labels after taking out ones with 5 or less items are", len(labels)) #56 labels

<ipython-input-21-f1a7a2d926e9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  krg_concatenated.dropna(subset=["narrative"], inplace=True)  #3366 rows, 265 column


(3366, 264)
(3362, 266)
labels after taking out ones with 5 or less items are 56


<ipython-input-21-f1a7a2d926e9>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  krg_concatenated['concatenated'] = krg_concatenated.apply(concatenate_values, axis=1)
<ipython-input-21-f1a7a2d926e9>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  krg_concatenated["labels"] = krg_narratives["labels"]
<ipython-input-21-f1a7a2d926e9>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

From the Karonga HDSS dataset we obtained 2 subsets:

* **krg_narratives**: contains the **narratives only**, and all the classes with 5 or less items were removed. 3362 rows × 2 columns, 56 labels
* **krg_concatenated**: subset of all items that contain **both the closed questions and the narratives**, and all the classes with 5 or less items were removed. 3362 rows × 265 columns, 56 labels


### Million Death Study dataset

In [ ]:
# ADULTS
mds_closed = pd.read_csv("/content/mds_rd1_adult_v1.csv") # shaped 95382, 124
mds_narratives = pd.read_csv("/content/mds_rd1_adult_narrative_v1.csv") # shaped 8484, 2
print(mds_closed.shape, mds_narratives.shape)

#only keep the VAs with narratives
mds = pd.merge(mds_closed, mds_narratives[['rowid', 'narrative']], on='rowid', how='inner')

# check if there are NaN values in the labels
print(mds['final_code'].isna().sum())
mds.shape #8484, 125

(95382, 124) (8484, 2)
0


(8484, 125)

In [ ]:
value_counts = mds['final_code'].value_counts()
print(value_counts)
values_to_remove = value_counts[value_counts <= 5].index
print(len(values_to_remove))
mds = mds[~mds["final_code"].isin(values_to_remove)]

mds['final_code'] = mds['final_code'].apply(lambda x: f"{x[0]}.{x[1]}.{x[2:]}") # modify the labels
mds.shape # (7884, 125)

final_code
I21    744
I64    506
R54    484
J45    469
A16    310
      ... 
I95      1
A03      1
A92      1
B97      1
C73      1
Name: count, Length: 482, dtype: int64
308


(7884, 125)

In [ ]:
# NARRATIVES ONLY DATASET
mds_narratives = mds[["narrative", "final_code"]]
mds_narratives # 7884 x 2

,narrative,final_code
0,Grand son of deceased Oma Devi Mr. ROMs Chande...,I.2.1
1,My wife Seema who expired on 30-9-02 was pregn...,O.7.2
2,It is the statement of sh Sakh Ram Husband of ...,R.5.0
3,Respondent said the deceased was pregnant and ...,O.1.5
4,Shaukat Ahmed was a laborer. On July 19 2002 h...,V.6.4
...,...,...
8479,The deceased was 17 years old when she died. T...,X.0.9
8480,Deceased name is Bieula & she got married in ...,O.7.2
8481,The deceased was in her 8th month of pregnancy...,O.8.8
8482,"At the age of nine years, the deceased was suf...",O.9.9


### 20newsgroup dataset

In [ ]:
# FOR TFIDF EMBEDDINGS
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train')
test  = fetch_20newsgroups(subset='test')

# X=lista di documenti, y=ndarray numerico
Xtr, ytr = train.data, train.target
Xte, yte = test.data, test.target

# trasformiamo i numeri (y) in etichette che hanno l'info della gerarchia
target_names = np.asarray(train.target_names)
ytr = target_names[ytr] # len 11314

target_names = np.asarray(test.target_names)
yte = target_names[yte]

In [ ]:
print(len(ytr), len(yte)) #18846

11314 7532


In [ ]:
# FOR PRETRAINED EMBEDDINGS
from sklearn.datasets import fetch_20newsgroups

twenty_news = fetch_20newsgroups(subset="all")
target_names = np.asarray(twenty_news.target_names)
labels = target_names[twenty_news.target]

df_20news = pd.DataFrame(zip(twenty_news.data, labels), columns=['text', 'label'])

value_counts = df_20news['label'].value_counts()
values_to_remove = value_counts[value_counts <= 2].index
df_20news = df_20news[~df_20news['label'].isin(values_to_remove)]

df_20news.shape

(18846, 2)

<a name="train_test"></a>
# 2. Embeddings and train-test split

#### Tfidf vectors

In [ ]:
# TEXT LABEL SPLIT
#X_krg_closed = krg_closed.values
#y_krg_closed= krg_closed["labels"].values

X_krg_narratives = krg_narratives["narrative"].values
y_krg_narratives = krg_narratives["labels"].values

#X_krg_concatenated = krg_concatenated["concatenated"].values
#y_krg_concatenated= krg_concatenated["labels"].values

#X_mds_narratives = mds_narratives["narrative"].values
#y_mds_narratives = mds_narratives["final_code"].values


In [ ]:
#Xtr, Xte, ytr, yte = train_test_split(X_krg_closed, y_krg_closed, test_size=0.2, stratify=y_krg_closed, random_state=0)
Xtr, Xte, ytr, yte = train_test_split(X_krg_narratives, y_krg_narratives, test_size=0.2, stratify=y_krg_narratives, random_state=0)
#Xtr, Xte, ytr, yte = train_test_split(X_krg_concatenated, y_krg_concatenated, test_size=0.2, stratify=y_krg_concatenated, random_state=0)

#Xtr, Xte, ytr, yte = train_test_split(X_mds_narratives, y_mds_narratives, test_size=0.2, stratify=y_mds_narratives, random_state=0)
#Xtr, Xte, ytr, yte = train_test_split(X_mds_concatenated, y_mds_concatenated, test_size=0.2, stratify=y_mds_concatenated, random_state=0)

In [ ]:
# TFIDF EMBEDDINGS
tfidf = TfidfVectorizer(min_df=5)
Xtr = tfidf.fit_transform(Xtr)
Xte = tfidf.transform(Xte)

In [ ]:
classes = sorted(np.unique(ytr))

train_data = LabelledCollection(Xtr, ytr, classes=classes)
test_data = LabelledCollection(Xte, yte, classes=classes)

#### Pretrained embeddings

In [ ]:
# PRETRAINED EMBEDDINGS
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
model = AutoModel.from_pretrained('distilbert-base-uncased')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_embeddings(text):
    tokenized = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="pt")
    with torch.no_grad():
        tokenized = tokenized.to(device)
        outputs = model(**tokenized)  # Move outputs to the same device as tokenized inputs
        embeddings = outputs.last_hidden_state
    # Move embeddings to the same device as tokenized inputs
    embeddings = embeddings.to(device)
    pooled_embeddings = torch.mean(embeddings, dim=1)
    return pooled_embeddings

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
#embeddings = get_embeddings(krg_narratives["narrative"].tolist())
#embeddings = get_embeddings(df_20news["text"].tolist())
embeddings = get_embeddings(mds_narratives["narrative"].tolist())

embeddings = embeddings.cpu().numpy()

In [ ]:
#TRAIN TEST SPLIT
#Xtr, Xte, ytr, yte = train_test_split(embedding_arrays, krg_narratives["labels"], test_size=0.2, stratify=krg_narratives["labels"], random_state=0)
Xtr, Xte, ytr, yte = train_test_split(embeddings, df_20news["label"], test_size=0.2, stratify= df_20news["label"], random_state=0)
#Xtr, Xte, ytr, yte = train_test_split(embeddings, mds_narratives["final_code"], test_size=0.2, stratify= mds_narratives["final_code"], random_state=0)

classes = sorted(np.unique(ytr))

train_data = LabelledCollection(Xtr, ytr, classes=classes)
test_data = LabelledCollection(Xte, yte, classes=classes)

# 3. Models

In [ ]:
timing_dictionary = {"model": [],
                     "mae_svm_train" : [],
                     "mae_svm_test" : [],
                     "mrae_svm_train" : [],
                     "mrae_svm_test" : [],

                     "mae_lr_train" : [],
                     "mae_lr_test" : [],
                     "mrae_lr_train" : [],
                     "mrae_lr_test" : []
                     }

<a name="1-2base"></a>
### 1° and 2° Baselines:

In [ ]:
def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result, elapsed_time
    return wrapper

@timer_decorator
def train_quantifier(model, train_data):
    return model.fit(train_data)

@timer_decorator
def test_quantifier(nhq, test_data, sample_size=100, repeats=100):
    # EVALUATION
    qp.environ['SAMPLE_SIZE'] = sample_size
    qp.environ['N_JOBS'] = -1

    # protocol = qp.protocol.NPP(test_data, repeats=100)
    protocol = qp.protocol.UPP(test_data, repeats=repeats, sample_size=sample_size)
    evaluation = qp.evaluation.evaluate(nhq, protocol=protocol, error_metric=metric)
    return evaluation

results1_dict = {"quantifier": [],
                "mae_svm": [],
                "mrae_svm": [],
                "mae_lr": [],
                "mrae_lr": []}

timing_dict = {"quantifier"}
for quantifier in [CC, ACC, PCC, EMQ]: # CC,PCC,ACC,PACC,EMQ
  results1_dict["quantifier"].append(quantifier.__name__)
  timing_dictionary["model"].append(quantifier.__name__)

  for classifier in [LinearSVC(), LogisticRegression(n_jobs=-1)]: # LinearSVC(), LogisticRegression(n_jobs=-1)

    for metric in ["mae", "mrae"]:
      model = quantifier(classifier)

      nhq, train_time = train_quantifier(model, train_data)
      evaluation, test_time = test_quantifier(nhq, test_data)

      print(quantifier.__name__, classifier.__class__.__name__, f"{metric} UPP evaluation: {evaluation:.3f}", f"train time: {train_time:.2f}", f"test time: {test_time:.2f}")

      if isinstance(classifier, LinearSVC):
        results1_dict[f"{metric}_svm"].append(round(evaluation, 3))
        timing_dictionary[f"{metric}_svm_train"].append(round(train_time, 2))
        timing_dictionary[f"{metric}_svm_test"].append(round(test_time, 2))
      else:
        results1_dict[f"{metric}_lr"].append(round(evaluation, 3))
        timing_dictionary[f"{metric}_lr_train"].append(round(train_time, 2))
        timing_dictionary[f"{metric}_lr_test"].append(round(test_time, 2))


CC LinearSVC mae UPP evaluation: 0.012 train time: 117.87 test time: 0.16
CC LinearSVC mrae UPP evaluation: 0.465 train time: 117.85 test time: 0.15
CC LogisticRegression mae UPP evaluation: 0.015 train time: 6.41 test time: 0.17
CC LogisticRegression mrae UPP evaluation: 0.581 train time: 6.03 test time: 0.15
ACC LinearSVC mae UPP evaluation: 0.014 train time: 223.50 test time: 2.83
ACC LinearSVC mrae UPP evaluation: 0.400 train time: 244.29 test time: 2.79
ACC LogisticRegression mae UPP evaluation: 0.017 train time: 13.55 test time: 2.54
ACC LogisticRegression mrae UPP evaluation: 0.485 train time: 12.53 test time: 2.50
PCC LinearSVC mae UPP evaluation: 0.017 train time: 475.38 test time: 0.19
PCC LinearSVC mrae UPP evaluation: 0.738 train time: 474.03 test time: 0.14
PCC LogisticRegression mae UPP evaluation: 0.017 train time: 6.96 test time: 0.16
PCC LogisticRegression mrae UPP evaluation: 0.730 train time: 5.92 test time: 0.15
EMQ LinearSVC mae UPP evaluation: 0.014 train time: 47

In [ ]:
# PACC LR ONLY

def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result, elapsed_time
    return wrapper

@timer_decorator
def train_quantifier(model, train_data):
    return model.fit(train_data)

@timer_decorator
def test_quantifier(nhq, test_data, sample_size=100, repeats=100):
    # EVALUATION
    qp.environ['SAMPLE_SIZE'] = sample_size
    qp.environ['N_JOBS'] = -1

    # protocol = qp.protocol.NPP(test_data, repeats=100)
    protocol = qp.protocol.UPP(test_data, repeats=repeats, sample_size=sample_size)
    evaluation = qp.evaluation.evaluate(nhq, protocol=protocol, error_metric=metric)
    return evaluation

data = []

for quantifier in [PACC]: # CC,PCC,ACC,PACC,EMQ
  results1_dict["quantifier"].append(quantifier.__name__)
  timing_dictionary["model"].append(quantifier.__name__)

  for classifier in [LogisticRegression(n_jobs=-1)]: # LinearSVC(), LogisticRegression(n_jobs=-1)

    for metric in ["mae", "mrae"]:
      model = quantifier(classifier)

      nhq, train_time = train_quantifier(model, train_data)
      evaluation, test_time = test_quantifier(nhq, test_data)

      print(quantifier.__name__, classifier.__class__.__name__, f"{metric} UPP evaluation: {evaluation:.3f}", f"train time: {train_time:.2f}", f"test time: {test_time:.2f}")

      results1_dict[f"{metric}_svm"].append("-")
      results1_dict[f"{metric}_lr"].append(round(evaluation, 3))

      timing_dictionary[f"{metric}_lr_train"].append(round(train_time, 2))
      timing_dictionary[f"{metric}_lr_test"].append(round(test_time, 2))
      timing_dictionary[f"{metric}_svm_train"].append("-")
      timing_dictionary[f"{metric}_svm_test"].append("-")

PACC LogisticRegression mae UPP evaluation: 0.014 train time: 12.04 test time: 3.24
PACC LogisticRegression mrae UPP evaluation: 0.390 train time: 11.98 test time: 3.17


<a name="3base"></a>
## 3° Baseline:

In [ ]:
from sklearn.base import BaseEstimator
import re
class HierarchicalClassifier(BaseEstimator):

    def __init__(self, classifier_creator=None, sep="."): # sep=r'(?<=\s)-|-(?=\s)'
        self.children = None
        self.node = classifier_creator()
        self.classifier_creator_ = classifier_creator
        self.classes_ = None
        self.level_classes_ = None
        self.sep_= sep

    def fit(self, X, y):

        self.classes_ = np.unique(y)

        # 1. GET THIS LEVEL
        this_level, other_level = self.get_this_level(y)
        if len(np.unique(this_level))==1:
          self.predict_class = this_level[0]
          return self


        # 1. FIT DEL CLASSIFIER
        self.node.fit(X, this_level)

        # GET THE CLASSES
        self.level_classes_ = np.unique(this_level)

        self.children = {}

        for class_name in self.level_classes_:
            sel_index = (this_level == class_name)
            X_child = X[sel_index]
            y_child = other_level[sel_index]

            n_classes = len(np.unique(y_child))
            if n_classes > 1:
                # if the children is not a leaf node, then we do the recurrent call
                child = HierarchicalClassifier(self.classifier_creator_,self.sep)
                self.children[class_name] = child.fit(X_child, y_child)
            else:
                # if the children is a leaf node, then we keep the rest of the string in place of a node;
                self.children[class_name] = y_child[0] # all classes have the same name, so we simply keep the first one

        return self

    @property
    def sep(self):
        return self.sep_

    def predict(self, X):
        if hasattr(self, 'predict_class'):
          return self.predict_class

        node_y = self.node.predict(X)

        predictions = []
        for x, y in zip(X, node_y):
            if isinstance(self.children[y], str):
                if len(self.children[y])==0:
                    predictions.append(y)
                else:
                    predictions.append(y+self.sep_+self.children[y])
            else:
                child_y = self.children[y].predict(x.reshape(1,-1))[0]
                predictions.append(y+self.sep_+child_y)

        return np.asarray(predictions)

    def get_this_level(self, y):
        this_level, other_level = [], []
        for yi in y:
            this, *other = yi.split(self.sep)
            other = self.sep.join(other)
            # print(f'{yi} : {this}  // {other}')
            this_level.append(this)
            other_level.append(other)
        this_level = np.asarray(this_level)
        other_level = np.asarray(other_level)
        return this_level, other_level

    def get_params(self, deep=True):
      # suppose this estimator has parameters "alpha" and "recursive"
      return {"classifier_creator": self.classifier_creator_, "sep": self.sep}

In [ ]:
def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result, elapsed_time
    return wrapper

@timer_decorator
def train_quantifier(model, Xtr, ytr):
    return model.fit(train_data)

@timer_decorator
def test_quantifier(model, Xte, yte, sample_size=100, repeats=100):
    qp.environ['SAMPLE_SIZE'] = sample_size
    qp.environ['N_JOBS'] = -1

    # protocol = qp.protocol.NPP(test_data, repeats=100)
    protocol = qp.protocol.UPP(test_data, repeats=repeats, sample_size=sample_size)
    evaluation = qp.evaluation.evaluate(model, protocol=protocol, error_metric=metric)
    return evaluation

#--------------------------------------------------------------------------
results3_dict = {"quantifier": [],
                "mae_svm": [],
                "mrae_svm": [],
                "mae_lr": [],
                "mrae_lr": []}

def C_LR():
    return LogisticRegression(n_jobs=-1)

def C_SVM():
    return LinearSVC()

for quantifier in [PACC]: #CC,ACC
  results3_dict["quantifier"].append(f"{quantifier.__name__}")
  timing_dictionary["model"].append(quantifier.__name__)

  for classifier_creator in [C_SVM, C_LR]:
    classifier_instance = classifier_creator()
    for metric in ["mae", "mrae"]:

      hc = HierarchicalClassifier(classifier_creator)
      model = quantifier(hc)
      model, train_time = train_quantifier(model, Xtr, ytr)
      evaluation, test_time = test_quantifier(model, Xte, yte)

      print(f'{quantifier.__name__, classifier_creator.__name__}')
      print(f'{metric}')
      print(f'{train_time=:.2f}s')
      print(f'{test_time=:.2f}s')
      print(f'{evaluation=:.3f}')

      if isinstance(classifier_instance, LinearSVC):
        results3_dict[f"{metric}_svm"].append(round(evaluation, 3))
        timing_dictionary[f"{metric}_svm_train"].append(round(train_time, 2))
        timing_dictionary[f"{metric}_svm_test"].append(round(test_time, 2))
      else:
        results3_dict[f"{metric}_lr"].append(round(evaluation, 3))
        timing_dictionary[f"{metric}_lr_train"].append(round(train_time, 2))
        timing_dictionary[f"{metric}_lr_test"].append(round(test_time, 2))

In [ ]:
results3_dict['quantifier']=["h(CC)", "h(ACC)"]
timing_dictionary["model"]=['CC', 'ACC', 'PCC', 'EMQ', 'PACC', "h(CC)", "h(ACC)"]

<a name="hier"></a>
## Hierarchical quantifier

### LabelledCollection, EmptySaveQuantifier

In [ ]:
# LABELLED COLLECTION, DATASET
from typing import Iterable

class LabelledCollection:
    """
    A LabelledCollection is a set of objects each with a label attached to each of them.
    This class implements several sampling routines and other utilities.

    :param instances: array-like (np.ndarray, list, or csr_matrix are supported)
    :param labels: array-like with the same length of instances
    :param classes: optional, list of classes from which labels are taken. If not specified, the classes are inferred
        from the labels. The classes must be indicated in cases in which some of the labels might have no examples
        (i.e., a prevalence of 0)
    """

    def __init__(self, instances, labels, classes=None):
        if issparse(instances):
            self.instances = instances
        elif isinstance(instances, list) and len(instances) > 0 and isinstance(instances[0], str):
            # lists of strings occupy too much as ndarrays (although python-objects add a heavy overload)
            self.instances = np.asarray(instances, dtype=object)
        else:
            self.instances = np.asarray(instances)
        self.labels = np.asarray(labels)
        n_docs = len(self)
        if classes is None:
            self.classes_ = np.unique(self.labels)
            self.classes_.sort()
        else:
            self.classes_ = np.unique(np.asarray(classes))
            self.classes_.sort()
            if len(set(self.labels).difference(set(classes))) > 0:
                raise ValueError(f'labels ({set(self.labels)}) contain values not included in classes_ ({set(classes)})')
        self.index = {class_: np.arange(n_docs)[self.labels == class_] for class_ in self.classes_}

    @classmethod
    def load(cls, path: str, loader_func: callable, classes=None, **loader_kwargs):
        """
        Loads a labelled set of data and convert it into a :class:`LabelledCollection` instance. The function in charge
        of reading the instances must be specified. This function can be a custom one, or any of the reading functions
        defined in :mod:`quapy.data.reader` module.

        :param path: string, the path to the file containing the labelled instances
        :param loader_func: a custom function that implements the data loader and returns a tuple with instances and
            labels
        :param classes: array-like, the classes according to which the instances are labelled
        :param loader_kwargs: any argument that the `loader_func` function needs in order to read the instances, i.e.,
            these arguments are used to call `loader_func(path, **loader_kwargs)`
        :return: a :class:`LabelledCollection` object
        """
        return LabelledCollection(*loader_func(path, **loader_kwargs), classes)

    def __len__(self):
        """
        Returns the length of this collection (number of labelled instances)

        :return: integer
        """
        return self.instances.shape[0]

    def prevalence(self):
        """
        Returns the prevalence, or relative frequency, of the classes in the codeframe.

        :return: a np.ndarray of shape `(n_classes)` with the relative frequencies of each class, in the same order
            as listed by `self.classes_`
        """
        return self.counts() / len(self)

    def counts(self):
        """
        Returns the number of instances for each of the classes in the codeframe.

        :return: a np.ndarray of shape `(n_classes)` with the number of instances of each class, in the same order
            as listed by `self.classes_`
        """
        return np.asarray([len(self.index[class_]) for class_ in self.classes_])

    @property
    def n_classes(self):
        """
        The number of classes

        :return: integer
        """
        return len(self.classes_)

    @property
    def binary(self):
        """
        Returns True if the number of classes is 2

        :return: boolean
        """
        return self.n_classes == 2

    def sampling_index(self, size, *prevs, shuffle=True, random_state=None):
        """
        Returns an index to be used to extract a random sample of desired size and desired prevalence values. If the
        prevalence values are not specified, then returns the index of a uniform sampling.
        For each class, the sampling is drawn with replacement.

        :param size: integer, the requested size
        :param prevs: the prevalence for each class; the prevalence value for the last class can be lead empty since
            it is constrained. E.g., for binary collections, only the prevalence `p` for the first class (as listed in
            `self.classes_` can be specified, while the other class takes prevalence value `1-p`
        :param shuffle: if set to True (default), shuffles the index before returning it
        :param random_state: seed for reproducing sampling
        :return: a np.ndarray of shape `(size)` with the indexes
        """
        if len(prevs) == 0:  # no prevalence was indicated; returns an index for uniform sampling
            return self.uniform_sampling_index(size, random_state=random_state)
        if len(prevs) == self.n_classes - 1:
            prevs = prevs + (1 - sum(prevs),)
        assert len(prevs) == self.n_classes, 'unexpected number of prevalences'
        assert sum(prevs) == 1, f'prevalences ({prevs}) wrong range (sum={sum(prevs)})'

        # Decide how many instances should be taken for each class in order to satisfy the requested prevalence
        # accurately, and the number of instances in the sample (exactly). If int(size * prevs[i]) (which is
        # <= size * prevs[i]) examples are drawn from class i, there could be a remainder number of instances to take
        # to satisfy the size constrain. The remainder is distributed along the classes with probability = prevs.
        # (This aims at avoiding the remainder to be placed in a class for which the prevalence requested is 0.)
        n_requests = {class_: round(size * prevs[i]) for i, class_ in enumerate(self.classes_)}
        remainder = size - sum(n_requests.values())
        with temp_seed(random_state):
            # due to rounding, the remainder can be 0, >0, or <0
            if remainder > 0:
                # when the remainder is >0 we randomly add 1 to the requests for each class;
                # more prevalent classes are more likely to be taken in order to minimize the impact in the final prevalence
                for rand_class in np.random.choice(self.classes_, size=remainder, p=prevs):
                    n_requests[rand_class] += 1
            elif remainder < 0:
                # when the remainder is <0 we randomly remove 1 from the requests, unless the request is 0 for a chosen
                # class; we repeat until remainder==0
                while remainder!=0:
                    rand_class = np.random.choice(self.classes_, p=prevs)
                    if n_requests[rand_class] > 0:
                        n_requests[rand_class] -= 1
                        remainder += 1

            indexes_sample = []
            for class_, n_requested in n_requests.items():
                n_candidates = len(self.index[class_])
                #print(n_candidates)
                #print(n_requested, 'rq')
                index_sample = self.index[class_][
                    np.random.choice(n_candidates, size=n_requested, replace=True)
                ] if n_requested > 0 else []

                indexes_sample.append(index_sample)

            indexes_sample = np.concatenate(indexes_sample).astype(int)

            if shuffle:
                indexes_sample = np.random.permutation(indexes_sample)

        return indexes_sample

    def uniform_sampling_index(self, size, random_state=None):
        """
        Returns an index to be used to extract a uniform sample of desired size. The sampling is drawn
        with replacement.

        :param size: integer, the size of the uniform sample
        :param random_state: if specified, guarantees reproducibility of the split.
        :return: a np.ndarray of shape `(size)` with the indexes
        """
        if random_state is not None:
            ng = RandomState(seed=random_state)
        else:
            ng = np.random
        return ng.choice(len(self), size, replace=True)

    def sampling(self, size, *prevs, shuffle=True, random_state=None):
        """
        Return a random sample (an instance of :class:`LabelledCollection`) of desired size and desired prevalence
        values. For each class, the sampling is drawn with replacement.

        :param size: integer, the requested size
        :param prevs: the prevalence for each class; the prevalence value for the last class can be lead empty since
            it is constrained. E.g., for binary collections, only the prevalence `p` for the first class (as listed in
            `self.classes_` can be specified, while the other class takes prevalence value `1-p`
        :param shuffle: if set to True (default), shuffles the index before returning it
        :param random_state: seed for reproducing sampling
        :return: an instance of :class:`LabelledCollection` with length == `size` and prevalence close to `prevs` (or
            prevalence == `prevs` if the exact prevalence values can be met as proportions of instances)
        """
        prev_index = self.sampling_index(size, *prevs, shuffle=shuffle, random_state=random_state)
        return self.sampling_from_index(prev_index)

    def uniform_sampling(self, size, random_state=None):
        """
        Returns a uniform sample (an instance of :class:`LabelledCollection`) of desired size. The sampling is drawn
        with replacement.

        :param size: integer, the requested size
        :param random_state: if specified, guarantees reproducibility of the split.
        :return: an instance of :class:`LabelledCollection` with length == `size`
        """
        unif_index = self.uniform_sampling_index(size, random_state=random_state)
        return self.sampling_from_index(unif_index)

    def sampling_from_index(self, index):
        """
        Returns an instance of :class:`LabelledCollection` whose elements are sampled from this collection using the
        index.

        :param index: np.ndarray
        :return: an instance of :class:`LabelledCollection`
        """
        documents = self.instances[index]
        labels = self.labels[index]
        return LabelledCollection(documents, labels, classes=self.classes_)

    def split_stratified(self, train_prop=0.6, random_state=None):
        """
        Returns two instances of :class:`LabelledCollection` split with stratification from this collection, at desired
        proportion.

        :param train_prop: the proportion of elements to include in the left-most returned collection (typically used
            as the training collection). The rest of elements are included in the right-most returned collection
            (typically used as a test collection).
        :param random_state: if specified, guarantees reproducibility of the split.
        :return: two instances of :class:`LabelledCollection`, the first one with `train_prop` elements, and the
            second one with `1-train_prop` elements
        """
        instances = self.instances
        labels = self.labels
        remainder = None
        for idx in np.argwhere(self.counts()==1):
            class_with_1 = self.classes_[idx.item()]
            if remainder is None:
                remainder = LabelledCollection(instances[labels==class_with_1], [class_with_1], classes=self.classes_)
            else:
                remainder += LabelledCollection(instances[labels==class_with_1], [class_with_1], classes=self.classes_)
            instances = instances[labels!=class_with_1]
            labels = labels[labels!=class_with_1]
        tr_docs, te_docs, tr_labels, te_labels = train_test_split(
            instances, labels, train_size=train_prop, stratify=labels, random_state=random_state
        )
        training = LabelledCollection(tr_docs, tr_labels, classes=self.classes_)
        test = LabelledCollection(te_docs, te_labels, classes=self.classes_)
        if remainder is not None:
            training += remainder
        return training, test

    def split_random(self, train_prop=0.6, random_state=None):
        """
        Returns two instances of :class:`LabelledCollection` split randomly from this collection, at desired
        proportion.

        :param train_prop: the proportion of elements to include in the left-most returned collection (typically used
            as the training collection). The rest of elements are included in the right-most returned collection
            (typically used as a test collection).
        :param random_state: if specified, guarantees reproducibility of the split.
        :return: two instances of :class:`LabelledCollection`, the first one with `train_prop` elements, and the
            second one with `1-train_prop` elements
        """
        indexes = np.random.RandomState(seed=random_state).permutation(len(self))
        if isinstance(train_prop, int):
            assert train_prop < len(self), \
                'argument train_prop cannot be greater than the number of elements in the collection'
            splitpoint = train_prop
        elif isinstance(train_prop, float):
            assert 0 < train_prop < 1, \
                'argument train_prop out of range (0,1)'
            splitpoint = int(np.round(len(self)*train_prop))
        left, right = indexes[:splitpoint], indexes[splitpoint:]
        training = self.sampling_from_index(left)
        test = self.sampling_from_index(right)
        return training, test

    def __add__(self, other):
        """
        Returns a new :class:`LabelledCollection` as the union of this collection with another collection.
        Both labelled collections must have the same classes.

        :param other: another :class:`LabelledCollection`
        :return: a :class:`LabelledCollection` representing the union of both collections
        """
        if not all(np.sort(self.classes_)==np.sort(other.classes_)):
            raise NotImplementedError(f'unsupported operation for collections on different classes; '
                                      f'expected {self.classes_}, found {other.classes_}')
        return LabelledCollection.join(self, other)

    @classmethod
    def join(cls, *args: Iterable['LabelledCollection']):
        """
        Returns a new :class:`LabelledCollection` as the union of the collections given in input.

        :param args: instances of :class:`LabelledCollection`
        :return: a :class:`LabelledCollection` representing the union of both collections
        """

        args = [lc for lc in args if lc is not None]
        assert len(args) > 0, 'empty list is not allowed for mix'

        assert all([isinstance(lc, LabelledCollection) for lc in args]), \
            'only instances of LabelledCollection allowed'

        first_instances = args[0].instances
        first_type = type(first_instances)
        assert all([type(lc.instances)==first_type for lc in args[1:]]), \
            'not all the collections are of instances of the same type'

        if issparse(first_instances) or isinstance(first_instances, np.ndarray):
            first_ndim = first_instances.ndim
            assert all([lc.instances.ndim == first_ndim for lc in args[1:]]), \
                'not all the ndarrays are of the same dimension'
            if first_ndim > 1:
                first_shape = first_instances.shape[1:]
                assert all([lc.instances.shape[1:] == first_shape for lc in args[1:]]), \
                    'not all the ndarrays are of the same shape'
            if issparse(first_instances):
                instances = vstack([lc.instances for lc in args])
            else:
                instances = np.concatenate([lc.instances for lc in args])
        elif isinstance(first_instances, list):
            instances = list(itertools.chain(lc.instances for lc in args))
        else:
            raise NotImplementedError('unsupported operation for collection types')
        labels = np.concatenate([lc.labels for lc in args])
        classes = np.unique(labels).sort()
        return LabelledCollection(instances, labels, classes=classes)

    @property
    def Xy(self):
        """
        Gets the instances and labels. This is useful when working with `sklearn` estimators, e.g.:

        >>> svm = LinearSVC().fit(*my_collection.Xy)

        :return: a tuple `(instances, labels)` from this collection
        """
        return self.instances, self.labels

    @property
    def Xp(self):
        """
        Gets the instances and the true prevalence. This is useful when implementing evaluation protocols from
        a :class:`LabelledCollection` object.

        :return: a tuple `(instances, prevalence)` from this collection
        """
        return self.instances, self.prevalence()

    @property
    def X(self):
        """
        An alias to self.instances

        :return: self.instances
        """
        return self.instances

    @property
    def y(self):
        """
        An alias to self.labels

        :return: self.labels
        """
        return self.labels

    @property
    def p(self):
        """
        An alias to self.prevalence()

        :return: self.prevalence()
        """
        return self.prevalence()


    def stats(self, show=True):
        """
        Returns (and eventually prints) a dictionary with some stats of this collection. E.g.,:

        >>> data = qp.datasets.fetch_reviews('kindle', tfidf=True, min_df=5)
        >>> data.training.stats()
        >>> #instances=3821, type=<class 'scipy.sparse.csr.csr_matrix'>, #features=4403, #classes=[0 1], prevs=[0.081, 0.919]

        :param show: if set to True (default), prints the stats in standard output
        :return: a dictionary containing some stats of this collection. Keys include `#instances` (the number of
            instances), `type` (the type representing the instances), `#features` (the number of features, if the
            instances are in array-like format), `#classes` (the classes of the collection), `prevs` (the prevalence
            values for each class)
        """
        ninstances = len(self)
        instance_type = type(self.instances[0])
        if instance_type == list:
            nfeats = len(self.instances[0])
        elif instance_type == np.ndarray or issparse(self.instances):
            nfeats = self.instances.shape[1]
        else:
            nfeats = '?'
        stats_ = {'instances': ninstances,
                  'type': instance_type,
                  'features': nfeats,
                  'classes': self.classes_,
                  'prevs': strprev(self.prevalence())}
        if show:
            print(f'#instances={stats_["instances"]}, type={stats_["type"]}, #features={stats_["features"]}, '
                  f'#classes={stats_["classes"]}, prevs={stats_["prevs"]}')
        return stats_

    def kFCV(self, nfolds=5, nrepeats=1, random_state=None):
        """
        Generator of stratified folds to be used in k-fold cross validation.

        :param nfolds: integer (default 5), the number of folds to generate
        :param nrepeats: integer (default 1), the number of rounds of k-fold cross validation to run
        :param random_state: integer (default 0), guarantees that the folds generated are reproducible
        :return: yields `nfolds * nrepeats` folds for k-fold cross validation
        """
        kf = RepeatedStratifiedKFold(n_splits=nfolds, n_repeats=nrepeats, random_state=random_state)
        for train_index, test_index in kf.split(*self.Xy):
            train = self.sampling_from_index(train_index)
            test = self.sampling_from_index(test_index)
            yield train, test

    def empty_classes(self):
        """
        Returns a np.ndarray of empty classes (classes present in self.classes_ but with
        no positive instance). In case there is none, then an empty np.ndarray is returned

        :return: np.ndarray
        """
        idx = np.argwhere(self.counts()==0).flatten()
        return self.classes_[idx]

    def non_empty_classes(self):
        """
        Returns a np.ndarray of non-empty classes (classes present in self.classes_ but with
        at least one positive instance). In case there is none, then an empty np.ndarray is returned

        :return: np.ndarray
        """
        idx = np.argwhere(self.counts() > 0).flatten()
        return self.classes_[idx]

    def has_empty_classes(self):
        """
        Checks whether the collection has empty classes

        :return: boolean
        """
        return len(self.empty_classes()) > 0

    def compact_classes(self):
        """
        Generates a new LabelledCollection object with no empty classes. It also returns a np.ndarray of
        indexes that correspond to the old indexes of the new self.classes_.

        :return: (LabelledCollection, np.ndarray,)
        """
        non_empty = self.non_empty_classes()
        all_classes = self.classes_
        old_pos = np.searchsorted(all_classes, non_empty)
        non_empty_collection = LabelledCollection(*self.Xy, classes=non_empty)
        return non_empty_collection, old_pos

class Dataset:
    """
    Abstraction of training and test :class:`LabelledCollection` objects.

    :param training: a :class:`LabelledCollection` instance
    :param test: a :class:`LabelledCollection` instance
    :param vocabulary: if indicated, is a dictionary of the terms used in this textual dataset
    :param name: a string representing the name of the dataset
    """

    def __init__(self, training: LabelledCollection, test: LabelledCollection, vocabulary: dict = None, name=''):
        assert set(training.classes_) == set(test.classes_), 'incompatible labels in training and test collections'
        self.training = training
        self.test = test
        self.vocabulary = vocabulary
        self.name = name

    @classmethod
    def SplitStratified(cls, collection: LabelledCollection, train_size=0.6):
        """
        Generates a :class:`Dataset` from a stratified split of a :class:`LabelledCollection` instance.
        See :meth:`LabelledCollection.split_stratified`

        :param collection: :class:`LabelledCollection`
        :param train_size: the proportion of training documents (the rest conforms the test split)
        :return: an instance of :class:`Dataset`
        """
        return Dataset(*collection.split_stratified(train_prop=train_size))

    @property
    def classes_(self):
        """
        The classes according to which the training collection is labelled

        :return: The classes according to which the training collection is labelled
        """
        return self.training.classes_

    @property
    def n_classes(self):
        """
        The number of classes according to which the training collection is labelled

        :return: integer
        """
        return self.training.n_classes

    @property
    def binary(self):
        """
        Returns True if the training collection is labelled according to two classes

        :return: boolean
        """
        return self.training.binary

    @classmethod
    def load(cls, train_path, test_path, loader_func: callable, classes=None, **loader_kwargs):
        """
        Loads a training and a test labelled set of data and convert it into a :class:`Dataset` instance.
        The function in charge of reading the instances must be specified. This function can be a custom one, or any of
        the reading functions defined in :mod:`quapy.data.reader` module.

        :param train_path: string, the path to the file containing the training instances
        :param test_path: string, the path to the file containing the test instances
        :param loader_func: a custom function that implements the data loader and returns a tuple with instances and
            labels
        :param classes: array-like, the classes according to which the instances are labelled
        :param loader_kwargs: any argument that the `loader_func` function needs in order to read the instances.
            See :meth:`LabelledCollection.load` for further details.
        :return: a :class:`Dataset` object
        """

        training = LabelledCollection.load(train_path, loader_func, classes, **loader_kwargs)
        test = LabelledCollection.load(test_path, loader_func, classes, **loader_kwargs)
        return Dataset(training, test)

    @property
    def vocabulary_size(self):
        """
        If the dataset is textual, and the vocabulary was indicated, returns the size of the vocabulary

        :return: integer
        """
        return len(self.vocabulary)

    @property
    def train_test(self):
        """
        Alias to `self.training` and `self.test`

        :return: the training and test collections
        :return: the training and test collections
        """
        return self.training, self.test

    def stats(self, show=True):
        """
        Returns (and eventually prints) a dictionary with some stats of this dataset. E.g.,:

        >>> data = qp.datasets.fetch_reviews('kindle', tfidf=True, min_df=5)
        >>> data.stats()
        >>> Dataset=kindle #tr-instances=3821, #te-instances=21591, type=<class 'scipy.sparse.csr.csr_matrix'>, #features=4403, #classes=[0 1], tr-prevs=[0.081, 0.919], te-prevs=[0.063, 0.937]

        :param show: if set to True (default), prints the stats in standard output
        :return: a dictionary containing some stats of this collection for the training and test collections. The keys
            are `train` and `test`, and point to dedicated dictionaries of stats, for each collection, with keys
            `#instances` (the number of instances), `type` (the type representing the instances),
            `#features` (the number of features, if the instances are in array-like format), `#classes` (the classes of
            the collection), `prevs` (the prevalence values for each class)
        """
        tr_stats = self.training.stats(show=False)
        te_stats = self.test.stats(show=False)
        if show:
            print(f'Dataset={self.name} #tr-instances={tr_stats["instances"]}, #te-instances={te_stats["instances"]}, '
                  f'type={tr_stats["type"]}, #features={tr_stats["features"]}, #classes={tr_stats["classes"]}, '
                  f'tr-prevs={tr_stats["prevs"]}, te-prevs={te_stats["prevs"]}')
        return {'train': tr_stats, 'test': te_stats}

    @classmethod
    def kFCV(cls, data: LabelledCollection, nfolds=5, nrepeats=1, random_state=0):
        """
        Generator of stratified folds to be used in k-fold cross validation. This function is only a wrapper around
        :meth:`LabelledCollection.kFCV` that returns :class:`Dataset` instances made of training and test folds.

        :param nfolds: integer (default 5), the number of folds to generate
        :param nrepeats: integer (default 1), the number of rounds of k-fold cross validation to run
        :param random_state: integer (default 0), guarantees that the folds generated are reproducible
        :return: yields `nfolds * nrepeats` folds for k-fold cross validation as instances of :class:`Dataset`
        """
        for i, (train, test) in enumerate(data.kFCV(nfolds=nfolds, nrepeats=nrepeats, random_state=random_state)):
            yield Dataset(train, test, name=f'fold {(i % nfolds) + 1}/{nfolds} (round={(i // nfolds) + 1})')


    def reduce(self, n_train=100, n_test=100):
        """
        Reduce the number of instances in place for quick experiments. Preserves the prevalence of each set.

        :param n_train: number of training documents to keep (default 100)
        :param n_test: number of test documents to keep (default 100)
        :return: self
        """
        self.training = self.training.sampling(n_train, *self.training.prevalence())
        self.test = self.test.sampling(n_test, *self.test.prevalence())
        return self

In [ ]:
import itertools


import quapy as qp
from quapy.method.base import BaseQuantifier
import numpy as np
from scipy.sparse import issparse
from scipy.sparse import vstack
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from numpy.random import RandomState
from quapy.functional import strprev
from quapy.util import temp_seed


class EmptySaveQuantifier(BaseQuantifier):
    def __init__(self, surrogate_quantifier: BaseQuantifier):
        self.surrogate_quantifier = surrogate_quantifier

    def fit(self, data: 'LabelledCollection'):
        self.n_classes = data.n_classes
        self.classes = data.classes_
        class_compact_data, self.old_class_idx = data.compact_classes()
        # print('OLD', self.old_class_idx)
        if self.num_non_empty_classes() > 1:
            self.surrogate_quantifier.fit(class_compact_data)
        else:
            self.surrogate_quantifier = TrivialQuantifier(data.classes_[0])
        return self

    def quantify(self, instances):
        if isinstance(self.surrogate_quantifier, TrivialQuantifier):
            return self.surrogate_quantifier.quantify(instances)

        num_instances = instances.shape[0]
        if self.num_non_empty_classes() == 0 or num_instances==0:
            # returns the uniform prevalence vector
            uniform = np.full(fill_value=1./self.n_classes, shape=self.n_classes, dtype=float)
            return uniform
        elif self.num_non_empty_classes() == 1:
            # returns a prevalence vector with 100% of the mass in the only non empty class
            prev_vector = np.full(fill_value=0., shape=self.n_classes, dtype=float)
            prev_vector[self.old_class_idx[0]] = 1
            return prev_vector
        else:
            class_compact_prev = self.surrogate_quantifier.quantify(instances)
            prev_vector = np.full(fill_value=0., shape=self.n_classes, dtype=float)
            prev_vector[self.old_class_idx] = class_compact_prev
            return prev_vector

    def num_non_empty_classes(self):
        return len(self.old_class_idx)

    def classify(self,X):
        if self.n_classes==1:
            n = X.shape[0]
            return np.asarray([classes[0]]*n)
        return self.surrogate_quantifier.classify(X)

    def crisp_classify(self, X):
        return self.surrogate_quantifier.classifier.predict(X)


class TrivialQuantifier():
    def __init__(self, label):
        self.label = label

    def fit(self, X, y):
        pass

    def quantify(self, X):
        return np.asarray([1.])


class HierarchicalQuantifier:

    def __init__(self, quantifier_creator=None, sep=".", root='root', final_classes=None):  # sep = r'(?<=\s)-|-(?=\s)'
        self.children = None
        self.node = quantifier_creator()
        self.quantifier_creator_ = quantifier_creator
        self.final_classes = final_classes
        self.sep_ = sep
        self.root = root

    def fit(self, X, y):

        # 1. GET THIS LEVEL
        this_level, other_level = self.get_this_level(y)

        # 1. FIT DEL QUANTIFIER
        this_level_train = LabelledCollection(X, this_level)
        self.classes_ = this_level_train.classes_
        self.internal_node = (this_level_train.n_classes)>1

        # print(f'Fit: root={self.root} is internal node={self.internal_node}')
        if not self.internal_node:
            assert other_level is None, 'unexpected other_level in leaf node'
            # return self

        self.node.fit(this_level_train)

        # GET THE CLASSES
        # print('fit: classes=', self.classes_)
        children_classes = np.unique(this_level)

        if other_level is None:
            self.internal_node = False

        else:
            # get the rest of the labels for other level
            self.children = {}
            for class_name in children_classes:
                sel_index = (this_level == class_name)
                X_child = X[sel_index]
                y_child = other_level[sel_index]

                n_classes = len(np.unique(y_child))
                #assert n_classes>1, f'error: n_classes=1 in node-root={self.root}'
                child = HierarchicalQuantifier(self.quantifier_creator_, self.sep, root=class_name)
                self.children[class_name] = child.fit(X_child, y_child)

        return self

    @property
    def sep(self):
        return self.sep_

    def quantify(self, X, p_su=1.0):

        prev_this = self.node.quantify(X)
        relative_prev = prev_this * p_su

        if not self.internal_node:
            return {label: prev for label, prev in zip(self.classes_, relative_prev)}

        final_prevalences = {}

        classifications = self.node.crisp_classify(X)
        for class_i, prev_i in zip(self.classes_, relative_prev):
            if prev_i>0:
                sel_index = (classifications==class_i)
                if sel_index.any():
                    Sigma_i = X[sel_index]
                    if class_i in self.children:
                        if hasattr(self.children[class_i], 'quantify'):
                            child_prevalences = self.children[class_i].quantify(Sigma_i, prev_i)
                        else:
                            # the children is not a quantifier but a label of the only class with positive examples
                            # below this level
                            unique_label = self.children[class_i]
                            print(class_i, unique_label, prev_i)
                            child_prevalences = {class_i+self.sep+unique_label: prev_i}
                        child_prevalences = {class_i+self.sep+key: val for key, val in child_prevalences.items()}
                        final_prevalences = {**final_prevalences, **child_prevalences}
            else:  # prev_i == 0
                continue

        if self.root == 'root':
            # the root is the only node that needs to create an array of prevalences with an order imposed by self.final_classes
            ordered_prevs = np.asarray([final_prevalences.get(class_, 0.) for class_ in self.final_classes])
            # it can happen that the total probability distributed is less than 1; this can happen when, for example,
            # an internal node assigns a prev >0 to a node for which no document is routed by the classifier.
            # to counter this we normalize by the sum
            prob_mass = ordered_prevs.sum()
            # if not np.isclose(prob_mass, 1.0): print('correction applied')
            ordered_prevs /= prob_mass
            return ordered_prevs
        else:
            # all other nodes return a dictionary of label:prevalence
            return final_prevalences

    def get_this_level(self, y):
        this_level, other_level = [], []
        for yi in y:
            this, *other = yi.split(self.sep)
            this_level.append(this)
            other = self.sep.join(other)
            other = str.strip(other)
            other_level.append(other)
        this_level = np.asarray(this_level)

        if all([label == '' for label in other_level]):
            # if all labels in other_level are empty, then the recursion should stop
            other_level = []  # mark other_level as empty
        else:
            # not all labels are empty; this means a new recursion will happen. In this case
            # the label (empty) needs to be replaced with something that indicates the class
            # from which this instance comes (otherwise all "empty" will look as belonging to
            # the same class, thus misleading the classifier)
            for i in range(len(other_level)):
                if other_level[i] == '':
                    other_level[i] = 'from'+this_level[i]
        if len(other_level)>0:
            assert len(other_level) == len(this_level), 'level splitting mismatch'
            other_level = np.asarray(other_level)
        else:
            other_level = None

        while len(np.unique(this_level))==1 and other_level is not None:
            # if this_level only contains 1 label, we merge the next label in the hiearchy,
            # e.g., 1.1.0, 1.1.0, 1.2.0, 1.2.9, -> (1.1, 1.1, 1.2, 1.2,) (0, 0, 0, 9)
            this_level_add, other_level = self.get_this_level(other_level)
            this_level = [self.sep.join([t,o]) for t,o in zip(this_level, this_level_add)]

        return this_level, other_level

### Hierarchical quantifier

In [ ]:
def Q_CC_SVM():
    return EmptySaveQuantifier(CC(LinearSVC()))

def Q_CC_LR():
    return EmptySaveQuantifier(CC(LogisticRegression(n_jobs=-1)))

def Q_ACC_SVM():
    return EmptySaveQuantifier(ACC(LinearSVC()))

def Q_ACC_LR():
    return EmptySaveQuantifier(ACC(LogisticRegression(n_jobs=-1)))

def Q_PCC_SVM():
  return EmptySaveQuantifier(PCC(LogisticRegression(n_jobs=-1)))

def Q_PCC_LR():
  return EmptySaveQuantifier(PCC(LinearSVC()))

def Q_PACC_SVM():
 return EmptySaveQuantifier(PACC(LinearSVC()))

def Q_PACC_LR():
  return EmptySaveQuantifier(PACC(LogisticRegression(n_jobs=-1)))

def Q_EMQ_SVM():
  return EmptySaveQuantifier(EMQ(LinearSVC()))

def Q_EMQ_LR():
  return EmptySaveQuantifier(EMQ(LogisticRegression(n_jobs=-1)))


def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        return result, elapsed_time
    return wrapper


@timer_decorator
def train_quantifier(hq:HierarchicalQuantifier, Xtr, ytr):
    return hq.fit(Xtr, ytr)


@timer_decorator
def test_quantifier(hq, Xte, yte, sample_size=100, repeats=100):
    # EVALUATION
    qp.environ['SAMPLE_SIZE'] = sample_size
    qp.environ['N_JOBS'] = -1

    # protocol = qp.protocol.NPP(test_data, repeats=100)
    protocol = qp.protocol.UPP(test_data, repeats=repeats, sample_size=sample_size)
    evaluation = qp.evaluation.evaluate(hq, protocol=protocol, error_metric=metric)
    return evaluation


result_h_dict = {'quantifier': [],
                "mae_svm": [],
                "mrae_svm" : [],
                "mae_lr":[],
                "mrae_lr" : [],
                }

pairs = [[Q_CC_SVM, Q_CC_LR], [Q_ACC_SVM, Q_ACC_LR], [Q_PCC_SVM, Q_PCC_LR], [Q_PACC_LR], [Q_EMQ_SVM, Q_EMQ_LR]]

for pair in pairs:
  if len(pair)==2:
    result_h_dict["quantifier"].append((f"{pair[0].__name__}", f"{pair[1].__name__}"))
    timing_dictionary["model"].append((f"{pair[0].__name__}", f"{pair[1].__name__}"))
  else:
    result_h_dict["quantifier"].append(f"{pair[0].__name__}")
    timing_dictionary["model"].append(f"{pair[0].__name__}")

  for node_quantifier_creator in pair :

    for metric in ["mae", "mrae"]:
      hq = HierarchicalQuantifier(node_quantifier_creator, final_classes=classes)

      hq, train_time = train_quantifier(hq, Xtr, ytr)
      evaluation, test_time = test_quantifier(hq, Xte, yte)

      print(f'{node_quantifier_creator}')
      print(f'{metric}')
      print(f'{train_time=:.2f}s')
      print(f'{test_time=:.2f}s')
      print(f'{evaluation=:.5f}')

      if node_quantifier_creator.__name__=="Q_PACC_LR":
        result_h_dict[f"{metric}_lr"].append(round(evaluation, 3))
        result_h_dict[f"{metric}_svm"].append("-")
        timing_dictionary[f"{metric}_lr_train"].append(round(train_time, 2))
        timing_dictionary[f"{metric}_lr_test"].append(round(test_time, 2))
        timing_dictionary[f"{metric}_svm_train"].append("-")
        timing_dictionary[f"{metric}_svm_test"].append("-")
        continue

      elif node_quantifier_creator==pair[0]:
        result_h_dict[f"{metric}_svm"].append(round(evaluation, 3))
        timing_dictionary[f"{metric}_svm_train"].append(round(train_time, 2))
        timing_dictionary[f"{metric}_svm_test"].append(round(test_time, 2))

      elif node_quantifier_creator==pair[1]:
        result_h_dict[f"{metric}_lr"].append(round(evaluation, 3))
        timing_dictionary[f"{metric}_lr_train"].append(round(train_time, 2))
        timing_dictionary[f"{metric}_lr_test"].append(round(test_time, 2))


<function Q_CC_SVM at 0x7963da714ee0>
mae
train_time=105.20s
test_time=0.44s
evaluation=0.02153
<function Q_CC_SVM at 0x7963da714ee0>
mrae
train_time=107.22s
test_time=0.45s
evaluation=0.71981
<function Q_CC_LR at 0x7963da715870>
mae
train_time=13.46s
test_time=0.50s
evaluation=0.02275
<function Q_CC_LR at 0x7963da715870>
mrae
train_time=12.48s
test_time=0.44s
evaluation=0.79330
<function Q_ACC_SVM at 0x7963da715900>
mae
train_time=198.52s
test_time=6.24s
evaluation=0.02247
<function Q_ACC_SVM at 0x7963da715900>
mrae
train_time=196.37s
test_time=6.13s
evaluation=0.61154
<function Q_ACC_LR at 0x7963da715990>
mae
train_time=23.90s
test_time=6.66s
evaluation=0.02421
<function Q_ACC_LR at 0x7963da715990>
mrae
train_time=26.87s
test_time=6.42s
evaluation=0.67353
<function Q_PCC_SVM at 0x7963da715a20>
mae
train_time=16.13s
test_time=0.46s
evaluation=0.02341
<function Q_PCC_SVM at 0x7963da715a20>
mrae
train_time=16.15s
test_time=0.43s
evaluation=0.88369
<function Q_PCC_LR at 0x7963da715d80>
m

In [ ]:
result_h_dict['quantifier']= ["H(CC)", "H(ACC)", "H(PCC)", "H(PACC)", "H(SLD)"]
timing_dictionary["model"]=["H(CC)", "H(ACC)", "H(PCC)", "H(PACC)", "H(SLD)" ]